In [407]:
import pandas as pd
import numpy as np
import datetime as dt
import re
import requests
import plotly.express as px
from plotly.subplots import make_subplots

In [143]:
dic_cols = {'country':'pais','type':'tipo','year':'anio','consumption':'cons','production':'produccion','gdp':'pbi','population':'poblacion','intensity':'intensidad','emission':'emision_co2','indicator':'indicador','fuel':'combustible','wind':'eolica','electricity':'elec','energy':'energia','iso_code':'pais_iso','share':'participacion','renewable':'renovable','oil':'petroleo','change':'cambio','coal':'carbon','time':'periodo','product':'producto','value':'valor','unit':'unidad','iso3':'pais_iso','source':'fuente',}

In [142]:
def nomCols(df,dic):
    for column in df.columns:
        df.rename(columns={column:column.lower()},inplace=True)
    
    for key in dic.keys():    
        for column in df.columns:
            if column == key:
                nom = column.replace(column,dic[key])
                df.rename(columns={column:nom},inplace=True)
            elif key in column:
                nom = column.replace(key,dic[key])
                df.rename(columns={column:nom},inplace=True)
    return df

def dropBeforeYear(df,year):
    lista = ['year','anio','año']
    for column in df.columns:
        if column.lower() in lista:
            df = df[df[column]>=year].reset_index(drop=True)
    return df

**EnergyCO2**

In [645]:
dataset = pd.read_csv(r"..\datasets\energyco2.csv")

In [646]:
df = pd.DataFrame(dataset)
df.head()

,Unnamed: 0,Country,Energy_type,Year,Energy_consumption,Energy_production,GDP,Population,Energy_intensity_per_capita,Energy_intensity_by_GDP,CO2_emission
0,0,World,all_energy_types,1980,292.899790,296.337228,27770.910281,4.298127e+06,68.145921,10.547,4946.627130
1,1,World,coal,1980,78.656134,80.114194,27770.910281,4.298127e+06,68.145921,10.547,1409.790188
2,2,World,natural_gas,1980,53.865223,54.761046,27770.910281,4.298127e+06,68.145921,10.547,1081.593377
3,3,World,petroleum_n_other_liquids,1980,132.064019,133.111109,27770.910281,4.298127e+06,68.145921,10.547,2455.243565
4,4,World,nuclear,1980,7.575700,7.575700,27770.910281,4.298127e+06,68.145921,10.547,0.000000


Dropeo la columna Unnamed: 0 que no contiene informacion

In [647]:
df = df.drop(columns="Unnamed: 0")

In [3]:
#Le paso una serie y un tipo para chequear si los elementos de la serie pertenecen a esa clase o no
def chequeoTipo(serie,clase):
    pertenece = False
    lista = []
    for item in serie:
        if isinstance(item,clase):
            pertenece = True
        else:
            lista.append(item)
    if len(lista)>0:
        return lista
    else: 
        return pertenece   


In [139]:
chequeoTipo(df['CO2_emission'],float)

NameError: name 'chequeoTipo' is not defined

Todos los features tienen elementos de las clases adecuadas para sus datos

Renombro columnas usando snake_case en español según definido por el equipo de trabajo

Una vez que ya chequee que cada columna tenga el tipo de dato adecuado, puedo usar esa condicion para chequear todas las columnas y convertir a float de dos digitos

In [648]:
for columna in df.columns:
    if isinstance(df[columna][1],float):
        df[columna] = df[columna].round(2)

<hr>

In [649]:
nomCols(df,dic_cols)

,pais,energia_tipo,anio,energia_cons,energia_produccion,pbi,poblacion,energia_intensidad_per_capita,energia_intensidad_by_pbi,co2_emision_co2
0,World,all_energy_types,1980,292.90,296.34,27770.91,4298126.52,68.15,10.55,4946.63
1,World,coal,1980,78.66,80.11,27770.91,4298126.52,68.15,10.55,1409.79
2,World,natural_gas,1980,53.87,54.76,27770.91,4298126.52,68.15,10.55,1081.59
3,World,petroleum_n_other_liquids,1980,132.06,133.11,27770.91,4298126.52,68.15,10.55,2455.24
4,World,nuclear,1980,7.58,7.58,27770.91,4298126.52,68.15,10.55,0.00
...,...,...,...,...,...,...,...,...,...,...
55435,Zimbabwe,coal,2019,0.05,0.08,37.62,14654.20,11.51,4.48,4.59
55436,Zimbabwe,natural_gas,2019,0.00,0.00,37.62,14654.20,11.51,4.48,0.00
55437,Zimbabwe,petroleum_n_other_liquids,2019,0.06,0.00,37.62,14654.20,11.51,4.48,4.38
55438,Zimbabwe,nuclear,2019,NaN,NaN,37.62,14654.20,11.51,4.48,0.00


In [650]:
df

,pais,energia_tipo,anio,energia_cons,energia_produccion,pbi,poblacion,energia_intensidad_per_capita,energia_intensidad_by_pbi,co2_emision_co2
0,World,all_energy_types,1980,292.90,296.34,27770.91,4298126.52,68.15,10.55,4946.63
1,World,coal,1980,78.66,80.11,27770.91,4298126.52,68.15,10.55,1409.79
2,World,natural_gas,1980,53.87,54.76,27770.91,4298126.52,68.15,10.55,1081.59
3,World,petroleum_n_other_liquids,1980,132.06,133.11,27770.91,4298126.52,68.15,10.55,2455.24
4,World,nuclear,1980,7.58,7.58,27770.91,4298126.52,68.15,10.55,0.00
...,...,...,...,...,...,...,...,...,...,...
55435,Zimbabwe,coal,2019,0.05,0.08,37.62,14654.20,11.51,4.48,4.59
55436,Zimbabwe,natural_gas,2019,0.00,0.00,37.62,14654.20,11.51,4.48,0.00
55437,Zimbabwe,petroleum_n_other_liquids,2019,0.06,0.00,37.62,14654.20,11.51,4.48,4.38
55438,Zimbabwe,nuclear,2019,NaN,NaN,37.62,14654.20,11.51,4.48,0.00


In [651]:
df_countries = df.query("pais!='World' & pais!='Germany, West' & pais!='Germany, East' & pais!='Former U.S.S.R.' & energia_tipo == 'all_energy_types'")

In [652]:
df_countries = df_countries.query('energia_tipo == "all_energy_types"')[['pais','anio','energia_cons','energia_produccion','co2_emision_co2']]
#df_countries_prod = df_countries.query('energia_tipo == "all_energy_types"').groupby(by='pais').sum()

In [653]:
df_energia = df_countries[['pais','energia_cons','energia_produccion']].groupby(by='pais').sum().reset_index().sort_values(by='energia_produccion',ascending=False).head(15)
df_energia['net_energy'] = df_energia['energia_produccion'] - df_energia['energia_cons']

In [654]:
paises = [pais for pais in df_energia['pais'].values]

In [656]:
fig = make_subplots()
fig.add_trace(go.Bar(x=df_energia['pais'],y=df_energia['energia_cons'],name='Consumo'))
fig.add_trace(go.Bar(x=df_energia['pais'],y=df_energia['energia_produccion'],base='stack',name='Produccion',opacity=0.6))
fig.add_trace(go.Scatter(x=df_energia['pais'],y=df_energia['net_energy'],name='Neto'))
fig.update_layout(go.Layout(
    barmode='overlay'
))
fig.update_layout(title='Consumo vs produccion en top paises productores energia',title_x = 0.4)
fig.show()

In [22]:
import plotly.express as px
import plotly.graph_objects as go

In [119]:
# fig = px.line(df_countries_cons.loc['Russia'],x=df_countries_cons.loc['Russia'].index, y='energia_cons' )
# fig2 = px.line(df_countries_cons.loc['China'],x=df_countries_cons.loc['China'].index, y='energia_cons' )
# fig3 = px.line(df_countries_cons.loc['United States'],x=df_countries_cons.loc['United States'].index, y='energia_cons' )

paises = ['Russia','China','United States', 'India']
# colores = ['red','green','blue','yellow']
# fig = ''

# for k,i in enumerate(paises):
#     fig += "px.line(df_countries_cons.loc['%s'],x=df_countries_cons.loc['%s'].index, y='energia_cons',color_discrete_sequence={'%s':px.colors.qualitative.Alphabet[%i]}).data +" % (i,i,i,k)


fig = px.line(df_countries_cons[[True if p in paises else False for p in df_countries_cons.pais]],x='anio',y='energia_cons',color='pais')
fig.update_layout(title='Lista paises',title_x=0.5)

<h1>Climate Related Disasters Frequency</h1>
<h2><i>Version CSV</i></h2>

In [725]:
dataset = pd.read_csv(r"..\datasets\desastres_naturales\Climate-related_Disasters_Frequency.csv")
df2 = pd.DataFrame(dataset)

Convierto NaN en ceros (en este caso se trata de frecuencias) y dropeo columnas que no me sirven

In [726]:
df2 = df2.drop(columns=['ObjectId','ISO2','Code','Unit','Source'])

Aplico la funcion para renombrar columnas nomCols usando el diccionario creado, dic_cols y con regex para sacar las "f" de las columnas de años. Aprovecho que itero sobre las columnas de tipo de dato float y aplico round(2)

In [727]:
nomCols(df2,dic_cols)

for column in df2.columns:
    if re.search('f\d{2,}', column):
        nom = column.split('f')[1]
        df2[column].fillna(0)
        df2[column]=df2[column].round(2)
        df2.rename(columns={column:nom},inplace=True)

Recorto los datos de la columna indicador para mostrar unicamente los descriptores

In [728]:
df2['indicador'] = df2['indicador'].apply(lambda x: x.split(': ')[1])

In [729]:
df2.loc[df2['pais_iso']=='CHN', 'pais'] = 'China'
df2.loc[df2['pais_iso']=='SAU', 'pais'] = 'Saudi Arabia'

In [730]:
cols=['pais','pais_iso','indicador']

df_climateDis = pd.melt(df2,id_vars=cols,var_name='anio',value_name='frecuencia')
df_climateDis =  df_climateDis.astype({'anio':'int','frecuencia':'float'})
df_climateDis = df_climateDis.merge(df_countries,on=['pais','anio'])

In [731]:
paises = [pais for pais in df_energia['pais'].head(5).values]

In [732]:
df_climateDis_top = df_climateDis[df_climateDis['pais'].isin(paises)]

In [733]:
df3 = df_climateDis_top[df_climateDis_top['indicador'] == 'TOTAL']

In [734]:
df3

,pais,pais_iso,indicador,anio,frecuencia,energia_cons,energia_produccion,co2_emision_co2
128,Canada,CAN,TOTAL,1980,1.0,9.55,10.15,NaN
149,China,CHN,TOTAL,1980,5.0,19.11,19.54,NaN
518,Saudi Arabia,SAU,TOTAL,1980,NaN,1.65,22.43,NaN
633,United States,USA,TOTAL,1980,7.0,78.02,67.15,4946.63
786,Canada,CAN,TOTAL,1981,1.0,9.42,9.96,422.55
...,...,...,...,...,...,...,...,...
25637,United States,USA,TOTAL,2018,19.0,101.20,95.75,5131.08
25790,Canada,CAN,TOTAL,2019,2.0,14.98,23.52,613.09
25811,China,CHN,TOTAL,2019,13.0,151.61,123.59,10608.60
26180,Saudi Arabia,SAU,TOTAL,2019,3.0,10.16,27.86,567.99


In [735]:
fig = px.line(df3,x='anio',y=['frecuencia','co2_emision_co2'],color='pais')
fig.show()

In [736]:
# fig1 = px.line(df3,x='anio', y=df3['frecuencia'].rolling(10).mean()*500,color='pais')
# fig2 = px.line(df3,x='anio', y='co2_emision_co2',color='pais')

# data = (fig1.data + fig2.data)

# fig = go.Figure(data=data)
# fig.show()

In [744]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

for pais in paises:
    fig.add_trace(go.Scatter(x=df3[df3['pais']==pais]['anio'],y=df3[df3['pais']==pais]['frecuencia'].rolling(10).mean(),name='Frecuencia '+pais))
    fig.add_trace(go.Scatter(x=df3[df3['pais']==pais]['anio'],y=df3[df3['pais']==pais]['co2_emision_co2'],name='CO2 '+pais),secondary_y=True)
fig.update_layout(title='Frecuencia de desastres climaticos vs produccion energía en top productores')
fig.show()

In [738]:
df5 = df3[df3['pais'] != 'India']

In [743]:
df5[['frecuencia','co2_emision_co2']].corr()

,frecuencia,co2_emision_co2
frecuencia,1.000000,0.830598
co2_emision_co2,0.830598,1.000000


In [740]:
df2.to_parquet(r"..\desastres_naturales\Climate-related_Disasters_Frequency_normalizado.parquet")

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

<hr>

<h1>OWID-Energy-Consumption-Source</h1>

In [ ]:
dataset = pd.read_csv(r"..\datasets\energy_consumption\owid-energy-consumption-source.csv")
df3 = pd.DataFrame(dataset)

In [ ]:
df3.head()

,country,year,iso_code,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
0,Afghanistan,1900,AFG,4832414.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1901,AFG,4879685.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1902,AFG,4935122.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1903,AFG,4998861.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1904,AFG,5063419.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for i in range(3,128):
    if chequeoTipo(df3.iloc[:,i],float):
        continue
    else:
        print(f"{df3.iloc[0:1,i]}")

NameError: name 'chequeoTipo' is not defined

Renombro columnas y me quedo con datos de 1980 en adelante

In [ ]:
nomCols(df3,dic_cols)
df3 = dropBeforeYear(df3,1980)

In [ ]:
for columna in df3.columns:
    if isinstance(df3[columna][2],float):
        df3[columna] = df3[columna].round(2)

In [ ]:
df3.shape

(12309, 128)

In [ ]:
df3.dropna(thresh=4)

,pais,anio,pais_iso,poblacion,pbi,biocombustible_cons_change_pct,biocombustible_cons_change_twh,biocombustible_cons_per_capita,biofuel_cons,biocombustible_elec_per_capita,...,solar_participacion_elec,solar_share_energia,eolica_cons_change_pct,eolica_cons_change_twh,wind_cons,eolica_elec_per_capita,eolica_electricity,eolica_energy_per_capita,eolica_share_elec,eolica_share_energy
0,Afghanistan,1980,AFG,13356500.0,1.532984e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1981,AFG,13171679.0,1.564534e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1982,AFG,12882518.0,1.598041e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1983,AFG,12537732.0,1.675533e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1984,AFG,12204306.0,1.707215e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12304,Zimbabwe,2017,ZWE,14236599.0,2.194784e+10,NaN,NaN,NaN,NaN,22.48,...,0.14,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
12305,Zimbabwe,2018,ZWE,14438812.0,2.271535e+10,NaN,NaN,NaN,NaN,27.01,...,0.11,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
12306,Zimbabwe,2019,ZWE,14645473.0,NaN,NaN,NaN,NaN,NaN,25.95,...,0.09,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
12307,Zimbabwe,2020,ZWE,14862927.0,NaN,NaN,NaN,NaN,NaN,24.22,...,0.09,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN


Elijo columnas que brindan informacion de cambio y 'per_capita' (no necesito esa info en el analisis planteado e igualmente puedo calcularlo con los demás datos de ser necesario)

In [ ]:
lista = []
for column in df3.columns:
    if '_pct' in column or 'cambio' in column or 'per_capita' in column or '_participacion_' in column:
        lista.append(column)

In [ ]:
df3 = df3.drop(columns=lista)

NameError: name 'lista' is not defined

In [ ]:
df3.head()

,pais,anio,pais_iso,poblacion,pbi,biocombustible_cons_change_pct,biocombustible_cons_change_twh,biocombustible_cons_per_capita,biofuel_cons,biocombustible_elec_per_capita,...,solar_participacion_elec,solar_share_energia,eolica_cons_change_pct,eolica_cons_change_twh,wind_cons,eolica_elec_per_capita,eolica_electricity,eolica_energy_per_capita,eolica_share_elec,eolica_share_energy
0,Afghanistan,1980,AFG,13356500.0,1.532984e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1981,AFG,13171679.0,1.564534e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1982,AFG,12882518.0,1.598041e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1983,AFG,12537732.0,1.675533e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1984,AFG,12204306.0,1.707215e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_consumo = df3.copy()

In [ ]:
df3.to_parquet(r"..\energy_consumption\owid-energy-consumption-source_normalizado.parquet")

<hr>

<h1>Energia Estadistica Mensual</h1>

In [ ]:
dataset = pd.read_csv(r"..\energia_estadistica_mensual\MES_0522.csv")
df4 = pd.DataFrame(dataset)

In [ ]:
df4.head(1)

,Country,Time,Balance,Product,Value,Unit
0,Australia,May 2022,Net Electricity Production,Electricity,21305.5276,GWh


In [ ]:
nomCols(df4,dic_cols)

,pais,periodo,balance,producto,valor,unidad
0,Australia,May 2022,Net Electricity Production,Electricity,21305.5276,GWh
1,Australia,May 2022,Net Electricity Production,Total Combustible Fuels,15280.3708,GWh
2,Australia,May 2022,Net Electricity Production,"Coal, Peat and Manufactured Gases",10488.3089,GWh
3,Australia,May 2022,Net Electricity Production,Oil and Petroleum Products,168.8215,GWh
4,Australia,May 2022,Net Electricity Production,Natural Gas,4388.9337,GWh
...,...,...,...,...,...,...
122896,IEA Total,January 2010,Total Imports,Electricity,34846.5580,GWh
122897,IEA Total,January 2010,Total Exports,Electricity,32372.5830,GWh
122898,IEA Total,January 2010,Used for pumped storage,Electricity,6869.2530,GWh
122899,IEA Total,January 2010,Distribution Losses,Electricity,62000.7330,GWh


In [ ]:
df4.shape

(122901, 6)

In [ ]:
df4.dropna(subset='valor',inplace=True)

In [ ]:
df4.isna().sum()

pais        0
periodo     0
balance     0
producto    0
valor       0
unidad      0
dtype: int64

In [ ]:
pertenece = False
lista = []
for item in df4.iloc[:,4]:
    if isinstance(item,float):
        pertenece = True
    else:
        lista.append((item,df4.columns[i]))
if len(lista)>0:
    print(lista)
else: 
    print(pertenece) 

True


In [ ]:
df4['periodo'] = pd.to_datetime(df4['periodo'])
df4['valor'] = df4['valor'].round(2)
df4['anio'] = df4['periodo'].dt.year
df4['mes'] = df4['periodo'].dt.month

In [ ]:
df4 = df4[['anio','mes','pais','balance','producto','valor']].sort_values(by=['anio','mes'])
df4.rename(columns={'valor':'cons_energia_gwh'},inplace=True)


In [ ]:
df4.to_parquet(r"C:\Users\ferch\Desktop\Henry\Proyecto Grupal\CO2-Emissions\datasets\energia_estadistica_mensual\MES_O522_normalizado.parquet")

<h1>Climate Related Disasters Frequency</h1>
<h2>Versión API</h2>

In [ ]:
url = "https://services9.arcgis.com/weJ1QsnbMYJlCHdG/arcgis/rest/services/Indicator_11_1_Physical_Risks_Climate_related_disasters_frequency/FeatureServer/0/query?where=1%3D1&outFields=Country,ISO3,Indicator,Source,F1980,F1981,F1982,F1983,F1984,F1985,F1986,F1987,F1988,F1989,F1990,F1991,F1992,F1993,F1994,F1995,F1996,F1997,F1998,F1999,F2000,F2001,F2002,F2003,F2004,F2005,F2006,F2007,F2008,F2009,F2010,F2011,F2012,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021&outSR=4326&f=json"
response = requests.get(url).json()

In [ ]:
response = requests.get(url).json()
df5 = pd.DataFrame()
for i in range(0,len(response['features'])):
    df = pd.DataFrame.from_dict([response['features'][i]['attributes']])
    data=(df5,df)
    df5=pd.concat(data)

In [ ]:
nomCols(df5,dic_cols)
df5.reset_index(drop=True,inplace=True)

Cambio nombres de columnas y aplico un fillna sobre los campos de frecuencias para luego aplicar un round

In [ ]:
for column in df5.columns:
    if re.search('f\d{2,}', column):
        df5[column] = df5[column].fillna(0)
        nom = column.split('f')[1]
        df5[column] = df5[column].round(2)
        df5.rename(columns={column:nom},inplace=True)

In [ ]:
df5['indicador'] = df2['indicador'].apply(lambda x: x.split(': ')[1])

In [ ]:
df6 = df5.groupby(by='pais').sum().transpose()

In [ ]:
df5.head()

,pais,pais_iso,indicador,fuente,1980,1981,1982,1983,1984,1985,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,"Afghanistan, Islamic Rep. of",AFG,Drought,"The Emergency Events Database (EM-DAT) , Centr...",0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
1,"Afghanistan, Islamic Rep. of",AFG,Extreme temperature,"The Emergency Events Database (EM-DAT) , Centr...",0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,"Afghanistan, Islamic Rep. of",AFG,Flood,"The Emergency Events Database (EM-DAT) , Centr...",1,0,0,0,0,0,...,6,4,2,1,4,1,3,6,5,2
3,"Afghanistan, Islamic Rep. of",AFG,Landslide,"The Emergency Events Database (EM-DAT) , Centr...",0,0,0,0,0,0,...,3,1,0,4,0,2,2,1,1,1
4,"Afghanistan, Islamic Rep. of",AFG,Storm,"The Emergency Events Database (EM-DAT) , Centr...",0,0,0,0,0,0,...,0,0,1,0,0,2,0,0,1,0


In [ ]:
dataset = pd.read_csv(r"C:\Users\ferch\Downloads\owid-energy-data.csv")
df7 = pd.DataFrame(dataset)

In [ ]:
dropBeforeYear(df7,1980)

,country,year,iso_code,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
0,Afghanistan,1980,AFG,13356500.0,1.532984e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1981,AFG,13171679.0,1.564534e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1982,AFG,12882518.0,1.598041e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1983,AFG,12537732.0,1.675533e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1984,AFG,12204306.0,1.707215e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12304,Zimbabwe,2017,ZWE,14236599.0,2.194784e+10,NaN,NaN,NaN,NaN,22.477,...,0.137,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
12305,Zimbabwe,2018,ZWE,14438812.0,2.271535e+10,NaN,NaN,NaN,NaN,27.011,...,0.110,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
12306,Zimbabwe,2019,ZWE,14645473.0,NaN,NaN,NaN,NaN,NaN,25.947,...,0.088,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
12307,Zimbabwe,2020,ZWE,14862927.0,NaN,NaN,NaN,NaN,NaN,24.221,...,0.090,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN


In [ ]:
for column in df7.columns:
    print(column)

iso_code
country
year
co2
co2_per_capita
trade_co2
cement_co2
cement_co2_per_capita
coal_co2
coal_co2_per_capita
flaring_co2
flaring_co2_per_capita
gas_co2
gas_co2_per_capita
oil_co2
oil_co2_per_capita
other_industry_co2
other_co2_per_capita
co2_growth_prct
co2_growth_abs
co2_per_gdp
co2_per_unit_energy
consumption_co2
consumption_co2_per_capita
consumption_co2_per_gdp
cumulative_co2
cumulative_cement_co2
cumulative_coal_co2
cumulative_flaring_co2
cumulative_gas_co2
cumulative_oil_co2
cumulative_other_co2
trade_co2_share
share_global_co2
share_global_cement_co2
share_global_coal_co2
share_global_flaring_co2
share_global_gas_co2
share_global_oil_co2
share_global_other_co2
share_global_cumulative_co2
share_global_cumulative_cement_co2
share_global_cumulative_coal_co2
share_global_cumulative_flaring_co2
share_global_cumulative_gas_co2
share_global_cumulative_oil_co2
share_global_cumulative_other_co2
total_ghg
ghg_per_capita
total_ghg_excluding_lucf
ghg_excluding_lucf_per_capita
methane
me

In [ ]:
from requests_html import HTMLSession, AsyncHTMLSession

In [ ]:
url = "https://github.com/owid/energy-data"

s = AsyncHTMLSession()
r = await s.get(url)

links = r.html.find('a')
for item in links:
    if item.text == 'CSV':
        link=item
        print(link)

In [ ]:
url = 'https://datacatalogapi.worldbank.org/ddhxext/ResourceFileData?resource_unique_id=DR0051731&version_id=2022-03-15T15:20:16.8826993Z'
response = requests.get(url).json()

In [ ]:
url = "https://www.climatewatchdata.org/api/v1/data/ndc_content/"
response = requests.get(url).json()
datos = response['data']

In [ ]:
for page in range(2, 1234):
    response = requests.get(url + f"?page={page}").json()
    datos.extend(response['data'])

In [ ]:
df = pd.DataFrame(datos)
df = df.drop(columns='id')

In [ ]:
df.overview_category.unique()

array(['Overall Comparison with Previous NDC', 'Summary of Commitment',
       'GHG Target, Summary of Commitment', 'UNFCCC Process',
       'Fairness and Ambition', 'NDC Enhancement Tracker', 'GHG Target',
       'Scope and Coverage', 'Overall Assumptions and Methodologies',
       'Accounting for Emissions and Removals from the Land Sector',
       'International Market Mechanism',
       'Overall Planning and Preparation', '',
       'Implementation (Adaptation)', 'Planning Process (Adaptation)',
       'Overview', 'Planning and Coordination Process',
       'Climate Change Trends and Impacts',
       'Adaptation vision and priorities', 'Financial Needs',
       'Experiences and Barriers', 'Monitoring and Evaluation',
       'Co-benefits', 'International Frameworks',
       'Gender and Indigenous People', 'Loss and Damage',
       'Transformative Adaptation', 'Finance and Support',
       'Planning Process', 'Broader Picture', 'Vulnerability',
       'Climate Risks', 'Adaptation Com

In [ ]:
df_summary = df.query("overview_category =='Summary of Commitment' & indicator_id =='ghg_target' & indicator_name == 'GHG target'")

In [ ]:
df_summary = df_summary.drop_duplicates()

In [ ]:
df_summary[df_summary['value'].str.contains('reduc')]

,source,iso_code3,country,global_category,overview_category,sector,subsector,indicator_id,indicator_name,value
18,CAIT,AFG,Afghanistan,Overview,Summary of Commitment,None,None,ghg_target,GHG target,13.6% reduction in GHG emissions by 2030 compa...
1063,CAIT,ALB,Albania,Overview,Summary of Commitment,None,None,ghg_target,GHG target,Reduce CO2 emissions compared to the baseline ...
1065,CAIT,ALB,Albania,Overview,Summary of Commitment,None,None,ghg_target,GHG target,Albania commits to reducing its emissions by 2...
3418,CAIT,DZA,Algeria,Overview,Summary of Commitment,None,None,ghg_target,GHG target,7% (with domestic means) to 22% (conditional) ...
4156,CAIT,AND,Andorra,Overview,Summary of Commitment,None,None,ghg_target,GHG target,37% (193.73 Gg CO2eq) reduction in 2030 compar...
...,...,...,...,...,...,...,...,...,...,...
59427,CAIT,DNK,Denmark,Overview,Summary of Commitment,None,None,ghg_target,GHG target,The European Union and its 28 Member States su...
59429,CAIT,DNK,Denmark,Overview,Summary of Commitment,None,None,ghg_target,GHG target,At least 55% reduction in greenhouse gas emiss...
60095,CAIT,DJI,Djibouti,Overview,Summary of Commitment,None,None,ghg_target,GHG target,40% (unconditional) to additional 20% (conditi...
61046,CAIT,DMA,Dominica,Overview,Summary of Commitment,None,None,ghg_target,GHG target,"17.9% by 2020, 39.2% by 2025 and 44.7% by 2030..."


In [ ]:
col = [pais, target, base, año_reduccion]

In [ ]:
df_summary.query("indicator_id =='ghg_target' & indicator_name == 'GHG target' & country ")

,source,iso_code3,country,global_category,overview_category,sector,subsector,indicator_id,indicator_name,value
0,CAIT,AFG,Afghanistan,Overview,Summary of Commitment,None,None,ghg_target,GHG target,13.6% reduction in GHG emissions by 2030 compa...
1,CAIT,ALB,Albania,Overview,Summary of Commitment,None,None,ghg_target,GHG target,Reduce CO2 emissions compared to the baseline ...
2,CAIT,ALB,Albania,Overview,Summary of Commitment,None,None,ghg_target,GHG target,Albania commits to reducing its emissions by 2...
3,CAIT,DZA,Algeria,Overview,Summary of Commitment,None,None,ghg_target,GHG target,7% (with domestic means) to 22% (conditional) ...
4,CAIT,AND,Andorra,Overview,Summary of Commitment,None,None,ghg_target,GHG target,37% (193.73 Gg CO2eq) reduction in 2030 compar...
...,...,...,...,...,...,...,...,...,...,...
90,CAIT,DNK,Denmark,Overview,Summary of Commitment,None,None,ghg_target,GHG target,The European Union and its 28 Member States su...
91,CAIT,DNK,Denmark,Overview,Summary of Commitment,None,None,ghg_target,GHG target,At least 55% reduction in greenhouse gas emiss...
92,CAIT,DJI,Djibouti,Overview,Summary of Commitment,None,None,ghg_target,GHG target,40% (unconditional) to additional 20% (conditi...
93,CAIT,DMA,Dominica,Overview,Summary of Commitment,None,None,ghg_target,GHG target,"17.9% by 2020, 39.2% by 2025 and 44.7% by 2030..."


In [ ]:
df_summary.iloc[2][9]

'Albania commits to reducing its emissions by 20.9% by 2030 compared to BAU.'

In [ ]:
for column in df_consumo.columns:
    print(column)

pais
anio
pais_iso
poblacion
pbi
biocombustible_cons_change_pct
biocombustible_cons_change_twh
biocombustible_cons_per_capita
biofuel_cons
biocombustible_elec_per_capita
biocombustible_electricity
biocombustible_share_elec
biocombustible_share_energy
carbon_intensidad_elec
coal_cons_cambio_pct
coal_cons_cambio_twh
carbon_cons_per_capita
coal_cons
carbon_elec_per_capita
coal_elec
coal_prod_cambio_pct
coal_prod_cambio_twh
carbon_prod_per_capita
coal_produccion
coal_participacion_elec
coal_share_energia
elec_demand
elec_generation
energia_cons_change_pct
energia_cons_change_twh
energia_per_capita
energy_per_pbi
fossil_cons_cambio_pct
fossil_cons_cambio_twh
fossil_elec_per_capita
fossil_elec
fossil_energia_per_capita
fossil_fuel_cons
fossil_participacion_elec
fossil_share_energia
gas_cons_cambio_pct
gas_cons_cambio_twh
gas_cons
gas_elec_per_capita
gas_elec
gas_energia_per_capita
gas_prod_cambio_pct
gas_prod_cambio_twh
gas_prod_per_capita
gas_produccion
gas_participacion_elec
gas_share_ener